In [1]:
import json
from models.retriever import retrieve_only

/home/ygong/miniconda3/envs/atlas-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
data_path = 'data/doc2dial/'
file_path = data_path  + 'doc2dial_dial_train.json'
with open(file_path, 'r') as f:
    data = json.load(f)

In [8]:
data['dial_data'].keys()

dict_keys(['dmv', 'ssa', 'va', 'studentaid'])

In [6]:
file_path = data_path + 'doc2dial_doc.json'
with open(file_path, 'r') as f:
    data = json.load(f)

In [4]:
len(data['doc_data']['ssa'].keys())

109

In [7]:
doc_list = list(data['doc_data']['ssa'].keys())

In [6]:
doc_list[0]

'Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1_0'

In [7]:
data['doc_data']['ssa'][doc_list[0]].keys()

dict_keys(['title', 'doc_id', 'domain', 'doc_text', 'spans', 'doc_html_ts', 'doc_html_raw'])

In [8]:
data['doc_data']['ssa'][doc_list[0]]['title']

'Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1'

In [13]:
data['doc_data']['ssa'][doc_list[0]]['spans']

{'1': {'id_sp': '1',
  'tag': 'h2',
  'start_sp': 0,
  'end_sp': 61,
  'text_sp': '\n\nBenefits Planner: Survivors | Planning For Your Survivors \n',
  'title': 'Benefits Planner: Survivors | Planning For Your Survivors',
  'parent_titles': [],
  'id_sec': 't_0',
  'start_sec': 0,
  'end_sec': 61,
  'text_sec': '\n\nBenefits Planner: Survivors | Planning For Your Survivors \n'},
 '2': {'id_sp': '2',
  'tag': 'u',
  'start_sp': 61,
  'end_sp': 90,
  'text_sp': 'As you plan for the future , ',
  'title': 'Benefits Planner: Survivors | Planning For Your Survivors',
  'parent_titles': [],
  'id_sec': '1',
  'start_sec': 61,
  'text_sec': "As you plan for the future , you'll want to think about what your family would need if you should die now. Social Security can help your family if you have earned enough Social Security credits through your work. ",
  'end_sec': 274},
 '3': {'id_sp': '3',
  'tag': 'u',
  'start_sp': 90,
  'end_sp': 168,
  'text_sp': "you'll want to think about what your f

In [25]:
data['doc_data']['ssa'][doc_list[0]].keys()

dict_keys(['title', 'doc_id', 'domain', 'doc_text', 'spans', 'doc_html_ts', 'doc_html_raw'])

In [8]:
def filter_and_write_to_json(data, keys, output_file) -> None:
    # Create a new dictionary with only the desired keys
    filtered_data = {key: data[key] for key in keys if key in data}

    # Write the filtered data to a JSON file
    with open(output_file, 'w') as file:
        json.dump(filtered_data, file)

In [9]:
keys_to_filter = ['title', 'doc_text']
filter_and_write_to_json(data['doc_data']['ssa'][doc_list[0]], keys_to_filter, 'data/doc1.json')

# VA

In [20]:
# data['doc_data']['va'].keys()
va_doc_list = list(data['doc_data']['va'].keys())

In [23]:
data['doc_data']['va'][va_doc_list[0]].keys()

dict_keys(['title', 'doc_id', 'domain', 'doc_text', 'spans', 'doc_html_ts', 'doc_html_raw'])

# Embedding

In [10]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/gtr-t5-base')

/home/ygong/miniconda3/envs/atlas-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
sentences = ["This is an example sentence", "Each sentence is converted"]
embeddings = model.encode(sentences)
print(embeddings)

[[-0.01774821  0.05172963  0.01236839 ... -0.00744141 -0.05631506
  -0.03496897]
 [-0.00638394  0.04372559  0.02519693 ... -0.00848611 -0.08687393
  -0.01826629]]


In [ ]:
def get_embeddings(batch, model):
    embeddings = model.encode(batch["text"])
    return {"embeddings": embeddings}

model = SentenceTransformer(
    "sentence-transformers/multi-qa-mpnet-base-dot-v1",
)

dataset = datasets.load_dataset("json", data_files="data.jsonl", split="train")
dataset = dataset.map(get_embeddings, batched=True, batch_size=32, fn_kwargs={"model": model})
dataset = dataset.with_format(
    type="numpy", columns=["embeddings"], output_all_columns=True,
)
dataset.add_faiss_index("embeddings")
dataset.save_faiss_index("embeddings", "index.faiss")

In [1]:
from langchain.document_loaders import JSONLoader

loader = JSONLoader(
    file_path='data/doc1.json',
    jq_schema='.doc_text',
)
document = loader.load()
print(f'documents:{len(document)}')

documents:1


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)
# remove empty documents
split_documents = text_splitter.split_documents(document)
print(f'documents:{len(split_documents)}')

documents:23


In [3]:
split_documents

[Document(page_content='Benefits Planner: Survivors | Planning For Your Survivors', metadata={'source': '/home/ygong/Attributed-QA/data/doc1.json', 'seq_num': 1}),
 Document(page_content="As you plan for the future , you'll want to think about what your family would need if you should die now. Social Security can help your family if you have earned enough Social Security credits through your work. You can earn up to four credits each year. In 2019 , for example , you earn one credit for each $1,360 of wages or self - employment income. When you have earned $5,440 , you have earned your four credits for the year. The number of credits needed to provide benefits for your survivors", metadata={'source': '/home/ygong/Attributed-QA/data/doc1.json', 'seq_num': 1}),
 Document(page_content="needed to provide benefits for your survivors depends on your age when you die. No one needs more than 40 credits 10 years of work to be eligible for any Social Security benefit. But , the younger a person 

In [4]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/gtr-t5-base')
# embeddings = SentenceTransformer('sentence-transformers/gtr-t5-base')

# docsearch = Chroma.from_documents(split_documents, embeddings, persist_directory="data/vector_store")
# docsearch.persist()

# docsearch = Chroma(persist_directory="data/vector_store", embedding_function=embeddings)

db = FAISS.from_documents(split_documents, embeddings)


/home/ygong/miniconda3/envs/atlas-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence_transformers`.

In [116]:
from langchain.vectorstores import Chroma
docsearch = Chroma.from_texts(split_documents, embeddings, metadatas=[{"source": str(i)} for i in range(len(split_documents))])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


AttributeError: 'Document' object has no attribute 'replace'

In [60]:
query = "who need 40 credits?"
docs = db.similarity_search(query)

In [16]:
db.save_local("data/faiss_index")

In [18]:
new_db = FAISS.load_local("data/faiss_index", embeddings)

In [113]:
docs

[Document(page_content="needed to provide benefits for your survivors depends on your age when you die. No one needs more than 40 credits 10 years of work to be eligible for any Social Security benefit. But , the younger a person is , the fewer credits they must have for family members to receive survivors benefits. Benefits can be paid to your children and your spouse who is caring for the children even if you don't have the required number of credits. They can get benefits if you have credit for one and one - half", metadata={'source': '/home/ygong/Attributed-QA/data/doc1.json', 'seq_num': 1}),
 Document(page_content='if you have credit for one and one - half years of work 6 credits in the three years just before your death.', metadata={'source': '/home/ygong/Attributed-QA/data/doc1.json', 'seq_num': 1}),
 Document(page_content="As you plan for the future , you'll want to think about what your family would need if you should die now. Social Security can help your family if you have e

In [1]:
paragraph = "Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles \"Crazy in Love\" and \"Baby Boy\"."

In [3]:
paragraph[207:]

'singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

# QA

In [32]:
from langchain import VectorDBQA
from langchain.chains import qa_with_sources
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = 'your_token'


In [64]:
# qa = qa_with_sources.from_chain_type(llm=qa_model, chain_type="map_reduce", vectorstore=db,return_source_documents=True)

qa_model = HuggingFaceHub(repo_id='google/flan-t5-base')
query = "Please answer the following question.\nHow need 40 credits?"
chain = load_qa_chain(llm=qa_model, chain_type="stuff")
chain.run(input_documents=docs, question=query, raw_response=True)


# result = qa({"query": "I'm looking for information regarding benefits planning, can you help me?"})
# print(result)

'No one needs more than 40 credits 10 years of work to be eligible for any Social Security benefit'

In [120]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
query = "Please answer the following question.\nHow need 40 credits?"
chain = load_qa_with_sources_chain(qa_model, chain_type="stuff")
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': 'No one needs more than 40 credits 10 years of work to be eligible for any Social Security benefit'}

In [56]:
docs

[Document(page_content='Benefits Planner: Survivors | Planning For Your Survivors', metadata={'source': '/home/ygong/Attributed-QA/data/doc1.json', 'seq_num': 1}),
 Document(page_content="If you have a surviving divorced spouse , they could get the same benefits as your widow or widower provided that your marriage lasted 10 years or more. Benefits paid to a surviving divorced spouse won't affect the benefit amounts your other survivors will receive based on your earnings record. If your former spouse is caring for your child who is under age 16 or disabled and gets benefits on your record , they will not have to meet the length - of - marriage rule. The child must be your", metadata={'source': '/home/ygong/Attributed-QA/data/doc1.json', 'seq_num': 1}),
 Document(page_content="on your record , they can switch to their own retirement benefit as early as age 62. This assumes they're eligible for retirement benefits and their retirement rate is higher than their rate as a widow, widower, o

In [66]:
# read from a text file
with open('api.txt', 'r') as file:
    api = file.read()

In [68]:
os.environ["OPENAI_API_KEY"] = api

In [114]:
from langchain import OpenAI
query = "Please answer the following question.\nHow need 40 credits?"
openai_model = OpenAI(model_name="text-davinci-003", max_tokens=1024)
chain = load_qa_with_sources_chain(openai_model, chain_type="stuff")
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

ImportError: Could not import openai python package. Please install it with `pip install openai`.

# evaluation

In [1]:
import evaluation
import pandas as pd

data_paths = ['data/doc2dial/result_1_cp500.csv', 'data/doc2dial/result_1.csv']

def read_data(data_paths):
    """Reads the data from the given paths."""
    dfs = []
    for path in data_paths:
        dfs.append(pd.read_csv(path))
    return pd.concat(dfs)

/home/ygong/miniconda3/envs/atlas-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = read_data(data_paths)
# df = df.dropna(subset=['answer'])

In [254]:
len(df)

21993

In [261]:
df.tail()

,question,answer,passage(context)
21487,If I want to know more about this topic and ma...,I want to know more about this topic and make ...,Making Payments
21488,ok i understand and what things should i avoid?,(iii),Circumstances That May Affect Your Grace Period
21489,What if it's not up to me and I can't make pay...,d).,I m having trouble making my loan payment. \nC...
21490,How can I make a payment by post?,You can mail a check or money order to a speci...,Making Payments
21491,Can you forgive the student loan?,in certain situations,Having Your Student Loan Forgiven \nYou are ge...


In [44]:
def load_doc_file(file_path):
    """
    Load doc2dial_doc.json file
    """
    #check if file is json
    if file_path.endswith('.json'):
        with open(file_path, 'r') as f:
            doc2dial_doc = json.load(f)
        return doc2dial_doc
    else:
        raise ValueError('File is not json')

In [45]:
doc_file_path = 'data/doc2dial/doc2dial_doc.json'
doc2dial_doc = load_doc_file(doc_file_path)

compute f1(model answer, true answer), autoais(q,a,ref), attribution_f1(retrieved answer, true answer)

In [9]:
qa_file_path = 'data/doc2dial/doc2dial_qa_train.csv'
qa_df = pd.read_csv(qa_file_path)

In [2]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration
# import torch

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# AUTOAIS = "google/t5_xxl_true_nli_mixture"
# hf_tokenizer = T5Tokenizer.from_pretrained(AUTOAIS)
# hf_model = T5ForConditionalGeneration.from_pretrained(AUTOAIS)
# if torch.cuda.device_count() > 1:
#     print('use multiple GPUs')
#     hf_model = torch.nn.DataParallel(hf_model)
# hf_model.to(device)

In [13]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

In [16]:
failed_doc_ids = [645,1448,1523,1573,8159]

def eval_test(df, doc2dial_doc, qa_df, failed_doc_ids, test_num=5):
    null_answer_cnt = 0
    tn = 0
    total_autoais = 0
    cnt = 0
    eval_df = pd.DataFrame(columns=['id', 'autoais', 'f1', 'att_f1'])

    for index, row in df.iterrows():
        print('index: ', index)
        print('question: ', row['question'])
        # print('test_num: ', tn)
        if tn == test_num:
            break
        tn += 1
        # print('index: ', index)
        # print('cnt: ', cnt)

        # qa_pair = qa_df.iloc[cnt]
        qa_pair = qa_df[qa_df['question'] == df.iloc[cnt]['question']].iloc[0]
        
        if row['question'] != qa_pair['question']:
            print('question not match')
            print('row question: ', row['question'])
            print('qa_pair question: ', qa_pair['question'])
            # cnt += 2
            # continue
        model_answer = row['answer']

        # if str(model_answer) == 'nan':
        #     print('answer is NaN')
        #     print('cnt: ', cnt)
        #     cnt += 1
        #     null_answer_cnt += 1
        #     continue
        retrived_doc = row['passage(context)']
        retrived_doc = retrived_doc.replace('/n', '\n')
        # print('retrived_doc: ', retrived_doc)

        sp_list = evaluation.get_ref(qa_pair, doc2dial_doc)
        ref_list = [item['text_sp'] for item in sp_list]
        true_ref = '\n'.join(ref_list)

        # print('true_ref: ', true_ref)

        #autoais
        example = {}
        example['question'] = row['question']
        example['answer'] = model_answer 
        example['passage'] = retrived_doc
            
        # print('model_answer: ', model_answer)
        print('true_ref: ', true_ref)
        print('true_answer', qa_pair['answer'])

        # autoais = evaluation.infer_autoais(example, hf_tokenizer, hf_model)
        # total_autoais += autoais == 'Y' 
        #f1 score
        try:
            f1 = evaluation.compute_f1(model_answer, qa_pair['answer'])
        except:
            print('cnt: ', cnt)
            null_answer_cnt += 1
        print('f1: ', f1)

        #attribution_f1 score (retrieved answer, true answer)
        att_f1 = evaluation.compute_f1(retrived_doc, true_ref)

        eval_df.loc[len(eval_df)] = [cnt, 'Y', f1, att_f1]
        if cnt == test_num:
            break
        cnt += 1

    print('null_answer_cnt: ', null_answer_cnt)
# ['avg', eval_df['autoais'].mean(), eval_df['f1'].mean(), eval_df['att_f1'].mean()]

In [17]:
eval_test(df, doc2dial_doc, qa_df, failed_doc_ids, test_num=5) #[ ] seems a bug here, first and second are both index 0

index:  0
question:  Hello, I forgot o update my address, can you help me with that?
true_ref:  you must report a change of address to DMV within ten days of moving. 
That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. 
true_answer hi, you have to report any change of address to DMV within 10 days after moving. You should do this both for the address associated with your license and all the addresses associated with all your vehicles.
f1:  0
index:  1
question:  Can I do my DMV transactions online?
true_ref:  Sign up or log into MyDMV [6 ] 
true_answer Yes, you can sign up for MyDMV for all the online transactions needed.
f1:  0
index:  2
question:  Thanks, and in case I forget to bring all of the documentation needed to the DMV office, what can I do?
true_ref:  About ten percent of customers visiting a DMV office do not bring what they need to complete their transaction, and have to come

In [187]:
qa_df[qa_df['question'] == df.iloc[1]['question']]

,question,answer,domain,doc_id,references,dial_id
1,Can I do my DMV transactions online?,"Yes, you can sign up for MyDMV for all the onl...",dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0,"[{'sp_id': '56', 'label': 'solution'}]",9f44c1539efe6f7e79b02eb1b413aa43


In [49]:
len(df)

21768

In [250]:
len(qa_df)

21998

In [210]:
#remove answer is NaN
df = df.dropna(subset=['answer'])
len(df)

21768

In [171]:
df.iloc[694]['question']

"I ordered my replacement license and still have not received it, it's over 2 weeks since I ordered it"

In [176]:
qa_pair = qa_df[qa_df['question'] == df.iloc[694]['question']]

In [201]:
qa_pair.iloc[0]['question']

"I ordered my replacement license and still have not received it, it's over 2 weeks since I ordered it"

In [181]:
qa_pair['question'] == df.iloc[694]['question']

695    True
Name: question, dtype: bool

In [243]:
df[1107:].head(10)

,question,answer,passage(context)
606,id like to register a moped,NaN,can certify a moped.
607,"no a regular moped, it has 2 wheels and moves ...",no,Register a moped[1] \n\n\nWhat is considered a...
608,i have a document i can bring to an office,yes,What do I need to bring to the DMV? \nBefore v...
609,i am in the UK where do i bring my documents?,DMV,What do I need to bring to the DMV? \nBefore v...
610,can i still use my moped without my documents?,no,What do I need to bring to the DMV? \nBefore v...
611,i will also bring proof of insurance and my id...,yes,to determine if you need insurance. Your insur...
612,i have full insurance and full drivers permit,no,permits and driver licenses. See information a...
613,I just passed my test and I want to know what ...,I want to know what to do next after I pass my...,If you fail your test \nDon't worry. If you fa...
614,What should I do after passing my road test if...,obey the restrictions for junior drivers,After you pass your test \nour license examine...
615,What happens if I fail my test?,(iv),If you fail your test \nDon't worry. If you fa...


In [229]:
qa_df[1653:].head(10)

,question,answer,domain,doc_id,references,dial_id
1653,Is there an additional fee?,Yes there's an annual fee and a renewal fee.,dmv,NYS Adventure License and Adventure Custom Pla...,"[{'sp_id': '7', 'label': 'solution'}]",1907fda3e3d0542cde4d39a67a77e871
1654,Will it take long?,You can pay a fee and get it immediately or wa...,dmv,NYS Adventure License and Adventure Custom Pla...,"[{'sp_id': '35', 'label': 'solution'}, {'sp_id...",1907fda3e3d0542cde4d39a67a77e871
1655,How much does it cost?,Some fees are waived for first timers.,dmv,NYS Adventure License and Adventure Custom Pla...,"[{'sp_id': '13', 'label': 'solution'}, {'sp_id...",1907fda3e3d0542cde4d39a67a77e871
1656,What do I do if I already have a lifetime spor...,Then you can request the icon be added to your...,dmv,NYS Adventure License and Adventure Custom Pla...,"[{'sp_id': '38', 'label': 'solution'}]",e2a552ecffdea8cdcd1cf2c488f4430d
1657,What do I do if I already have a lifetime spor...,Do you need to know what to expect at the DMV ...,dmv,NYS Adventure License and Adventure Custom Pla...,"[{'sp_id': '39', 'label': 'precondition'}]",e2a552ecffdea8cdcd1cf2c488f4430d
1658,"Yes, I've never done this before.",Bring the required proof and there is a $12.50...,dmv,NYS Adventure License and Adventure Custom Pla...,"[{'sp_id': '40', 'label': 'solution'}]",e2a552ecffdea8cdcd1cf2c488f4430d
1659,Are custom plates available now?,"Yes, the I LOVE NYS Adventure plates are curre...",dmv,NYS Adventure License and Adventure Custom Pla...,"[{'sp_id': '3', 'label': 'solution'}, {'sp_id'...",e2a552ecffdea8cdcd1cf2c488f4430d
1660,Can I order them online?,"No, but you can bring a copy of your driver's ...",dmv,NYS Adventure License and Adventure Custom Pla...,"[{'sp_id': '21', 'label': 'solution'}, {'sp_id...",e2a552ecffdea8cdcd1cf2c488f4430d
1661,Can you get these plates for any vehicle?,"Yes, they are available for passenger, commerc...",dmv,NYS Adventure License and Adventure Custom Pla...,"[{'sp_id': '3', 'label': 'solution'}, {'sp_id'...",e2a552ecffdea8cdcd1cf2c488f4430d
1662,Is it yearly or is there a lifetime option?,There is a Lifetime Empire Passport and you au...,dmv,NYS Adventure License and Adventure Custom Pla...,"[{'sp_id': '35', 'label': 'solution'}, {'sp_id...",e2a552ecffdea8cdcd1cf2c488f4430d


In [113]:
str(df.iloc[345]['answer']) == 'nan'

True

In [34]:
df.iloc[1]['passage(context)']

'About ten percent of customers visiting a DMV office do not bring what they need to complete their transaction, and have to come back a second time to finish their business. This can be as simple as not bringing sufficient funds to pay for a license renewal or not having the proof of auto insurance required to register a car. Better yet , don t visit a DMV office at all, and see if your transaction can be performed online, like an address change, registration renewal, license renewal, replacing'

In [24]:
qa_pair

question      Hello, I forgot o update my address, can you h...
answer        hi, you have to report any change of address t...
domain                                                      dmv
doc_id             Top 5 DMV Mistakes and How to Avoid Them#3_0
references    [{'sp_id': '6', 'label': 'solution'}, {'sp_id'...
dial_id                        9f44c1539efe6f7e79b02eb1b413aa43
Name: 0, dtype: object

In [30]:
evaluation.get_ref(qa_pair, doc2dial_doc)

[{'id_sp': '6',
  'tag': 'u',
  'start_sp': 346,
  'end_sp': 416,
  'text_sp': 'you must report a change of address to DMV within ten days of moving. ',
  'title': '1. Forgetting to Update Address',
  'parent_titles': [],
  'id_sec': '2',
  'start_sec': 333,
  'text_sec': 'By statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. ',
  'end_sec': 567},
 {'id_sp': '7',
  'tag': 'u',
  'start_sp': 416,
  'end_sp': 567,
  'text_sp': 'That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. ',
  'title': '1. Forgetting to Update Address',
  'parent_titles': [],
  'id_sec': '2',
  'start_sec': 333,
  'text_sec': 'By statute , you must report a change of address to DMV within ten days of moving. That is the case for the address

In [34]:
sp_list = evaluation.get_ref(qa_pair, doc2dial_doc)
ref_list = [item['text_sp'] for item in sp_list]
print('\n'.join(ref_list))

you must report a change of address to DMV within ten days of moving. 
That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. 


In [1]:
csv_file = 'data/doc2dial/qa_train_dmv.csv'
'doc2dial' in csv_file

True

# Test retrieve_only

In [4]:
from models.retriever import retrieve_only
import os

In [2]:
data_path = 'data/doc2dial/qa_train_dmv.csv'
result_df, failed_doc_ids = retrieve_only(data_path, cs=500, c_overlap=0, new_method=False)

search failed at:  431
search failed at:  1008
search failed at:  1028


In [3]:
result_df.head()

,question,answer,ref,passage(context),doc_id,dial_id
0,"Hello, I forgot o update my address, can you h...","hi, you have to report any change of address t...","[{'sp_id': '6', 'label': 'solution'}, {'sp_id'...",1. Forgetting to Update Address,Top 5 DMV Mistakes and How to Avoid Them#3_0,9f44c1539efe6f7e79b02eb1b413aa43
1,Can I do my DMV transactions online?,"Yes, you can sign up for MyDMV for all the onl...","[{'sp_id': '56', 'label': 'solution'}]",About ten percent of customers visiting a DMV ...,Top 5 DMV Mistakes and How to Avoid Them#3_0,9f44c1539efe6f7e79b02eb1b413aa43
2,"Thanks, and in case I forget to bring all of t...",This happens often with our customers so that'...,"[{'sp_id': '49', 'label': 'solution'}, {'sp_id...",About ten percent of customers visiting a DMV ...,Top 5 DMV Mistakes and How to Avoid Them#3_0,9f44c1539efe6f7e79b02eb1b413aa43
3,"Ok, and can you tell me again where should I r...",Sure. Any change of address must be reported t...,"[{'sp_id': '6', 'label': 'solution'}, {'sp_id'...","By statute , you must report a change of addre...",Top 5 DMV Mistakes and How to Avoid Them#3_0,9f44c1539efe6f7e79b02eb1b413aa43
4,Can you tell me more about Traffic points and ...,Traffic points is the system used by DMV to tr...,"[{'sp_id': '41', 'label': 'solution'}, {'sp_id...",4. Understanding how Much Traffic Points Cost,Top 5 DMV Mistakes and How to Avoid Them#3_0,9f44c1539efe6f7e79b02eb1b413aa43


In [6]:
#write to csv
#[x] mkdir ...
#[x] rename the file name, with info
fold_path = 'data/doc2dial'
if not os.path.exists(fold_path):
    os.makedirs(result_path)

result_path = os.path.join(fold_path, 'qa_train_dmv_withRefs.csv')
result_df.to_csv('data/doc2dial/qa_train_dmv_withRefs.csv', index=False)

In [29]:
save_dir = 'data/doc2dial/DEFAULT'
output = save_dir.split('/')[-1]
output = output+'_withRefs.csv'
output = save_dir + '/' + output

In [31]:
output.replace('withRefs', 'withModelAnswer')

'data/doc2dial/DEFAULT/DEFAULT_withModelAnswer.csv'

In [20]:
os.getcwd() + '/data/doc2dial/test.csv'

'/home/ygong/Attributed-QA/data/doc2dial/test.csv'

In [8]:
from configparser import ConfigParser

In [9]:
config_object = ConfigParser()

In [14]:
config_object['DEFAULT']
config_object.read("config.ini")
userinfo = config_object["DEFAULT"]
userinfo['chunk_size']

'500'

In [11]:
#read csv
import pandas as pd

df1 = pd.read_csv('data/doc2dial/doc2dial_testset/DEFAULT/DEFAULT_withModelAnswer.csv')
# df2 = pd.read_csv('data/doc2dial/doc2dial_testset/DEFAULT/DEFAULT_withRefs.csv')

In [12]:
df1.loc[0]

question          What can I do if I forgot to update my address?
answer          Don't forget to that, there are branches you c...
model_answer    You can update your address on your phone or c...
ref             [[{'sp_id': '5', 'label': 'solution'}, {'sp_id...
retrived_doc                      1. Forgetting to Update Address
doc_id               Top 5 DMV Mistakes and How to Avoid Them#3_0
Name: 0, dtype: object

In [13]:
df1.head()

,question,answer,model_answer,ref,retrived_doc,doc_id
0,What can I do if I forgot to update my address?,"Don't forget to that, there are branches you c...",You can update your address on your phone or c...,"[[{'sp_id': '5', 'label': 'solution'}, {'sp_id...",1. Forgetting to Update Address,Top 5 DMV Mistakes and How to Avoid Them#3_0
1,What if iI already moved?##Please go to a loca...,Please go to a local branch and update your ad...,ii.,"[[{'sp_id': '35', 'label': 'solution'}, {'sp_i...","By statute , you must report a change of addre...",Top 5 DMV Mistakes and How to Avoid Them#3_0
2,What if my insurance lapsed too?,You'll need to either bring a copy of your cur...,I would have to pay for the plates,"[[{'sp_id': '24', 'label': 'solution'}, {'sp_i...",possible mail correspondence can reach you. Al...,Top 5 DMV Mistakes and How to Avoid Them#3_0
3,I'll make sure to do so.,"If you need the forms mailed to you, or to dow...",b).,"[[{'sp_id': '29', 'label': 'solution'}]]",possible mail correspondence can reach you. Al...,Top 5 DMV Mistakes and How to Avoid Them#3_0
4,I'll make sure to do so.,Thank you; it will save you at least an hour i...,b).,"[[{'sp_id': '35', 'label': 'solution'}, {'sp_i...",possible mail correspondence can reach you. Al...,Top 5 DMV Mistakes and How to Avoid Them#3_0


In [9]:
new_df = pd.read_csv('data/doc2dial/qa_test_dmv.csv')

In [10]:
new_df.loc[0]

question          What can I do if I forgot to update my address?
answer          Don't forget to that, there are branches you c...
model_answer    You can update your address on your phone or c...
ref             [[{'sp_id': '5', 'label': 'solution'}, {'sp_id...
retrived_doc                      1. Forgetting to Update Address
doc_id               Top 5 DMV Mistakes and How to Avoid Them#3_0
Name: 0, dtype: object

In [46]:
from evaluation import get_ref
doc = df1.loc[0]
get_ref(doc, doc2dial_doc)

KeyError: 'references'

In [36]:
len(df2)

4147

In [41]:
import configparser
config = configparser.ConfigParser()
config.read("config.ini")

# Iterate over each section in the configuration file
for section in config.sections():
    print(f"Section: {section}")
    print() 

Section: doc2dial_500_top5

Section: doc2dial_500_top1_new



In [1]:
l = [1,2,3,4,5,6,7,8,9,10]
[i for i in l]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Test t5base

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from torch.utils.data import DataLoader
from document_loader import doc2dialDataset
from QAModel import local_answer_model
from models.retriever import retrieve_only
from document_loader import doc2dialEvalDataset
import ast
import evaluation

/home/ygong/miniconda3/envs/atlas-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
qa_model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(qa_model_name).to('cuda')

RuntimeError: CUDA out of memory. Tried to allocate 40.00 MiB (GPU 0; 10.92 GiB total capacity; 10.29 GiB already allocated; 11.38 MiB free; 10.29 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [2]:
fid_model = AutoModelForSeq2SeqLM.from_pretrained("Intel/fid_flan_t5_base_nq").to('cuda')
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

In [2]:
data_path = 'data/doc2dial/qa_test_dmv.csv'
# data_path = 'data/doc2dial/qa_train_dmv.csv'

result_df = retrieve_only(data_path, 
            cs=500, 
            c_overlap=0, 
            save_dir=None,
            new_method=False,
            embedding_model='sentence-transformers/gtr-t5-base',
            topk=1,
            test_mode=True)

dataset = doc2dialDataset(result_df)
dataloader = DataLoader(dataset, batch_size=8, shuffle=False)

first_batch = next(iter(dataloader))

In [71]:
import random

split_size = int(0.2 * len(dataset))
[random.randint(0, len(dataset)) for _ in range(split_size)]

[1, 7]

In [72]:
# choose 20% random samples from dataset
import random

mask = [random.randint(0, len(dataset)) for _ in range(split_size)]
subset = DataLoader(dataset, batch_size=2, shuffle=False, sampler=mask)


In [77]:
mask

[6, 1]

In [76]:
first_batch = next(iter(subset))
first_batch['question']

['What will happen if I let my insurance lapse?',
 'What if iI already moved?\nPlease go to a local branch with a piece of mail and update your address.']

In [4]:
questions = first_batch['question']
answers = first_batch['answer']
retrieve_docs = first_batch['retrived_doc']
refs = first_batch['ref']

retrieve_docs = first_batch['retrived_doc']
doc_ids = first_batch['doc_id']
dial_ids = first_batch['dial_id']

# device = 'cuda'

# model_answers = local_answer_model(fid_model, tokenizer, questions, retrieve_docs, device)
# print(model_answers)

In [6]:
questions

['What can I do if I forgot to update my address?',
 'What if iI already moved?\nPlease go to a local branch with a piece of mail and update your address.',
 'What if my insurance lapsed too?',
 "I'll make sure to do so.",
 "I'll make sure to do so.",
 "What if it doesn't solve my problem though?",
 'What will happen if I let my insurance lapse?',
 'No, my insurance is not lapsed right now.\nHow do I learn more about insurance lapses and how to surrender my plates?']

In [7]:
answers

["Don't forget to that, there are branches you can go to do this.\nPlease refrence local Maps or our website to find a local branch.",
 'Please go to a local branch and update your address.',
 "You'll need to either bring a copy of your current, new, insurance or pay the un-insured fee.\nPlease make sure to bring all documentation needed to a local branch to expedite this process.",
 'If you need the forms mailed to you, or to download them, please check our website.',
 'Thank you; it will save you at least an hour in line.',
 'Provided you bring all relevant documentation to the DMV branch, you should have no issues.',
 'It is a good thing that to not let your insurance lapse because otherwise, we all pay for it indirectly.  \nDo you know what happens when your insurance lapse?',
 'You have to contact the DMV if you want to surrender your plates and ask them how to do that.']

In [25]:
len(retrieve_docs[1])

496

In [3]:
# path = 'data/doc2dial/TEST/test.csv'
path = 'data/doc2dial/doc2dial_testset/doc2dial_250_top5/doc2dial_250_top5_withModelAnswer.csv'

dataset = doc2dialEvalDataset(path)
# inference task, so shuffle is False
dataloader = DataLoader(dataset, batch_size=16, shuffle=False)

first_batch = next(iter(dataloader))
questions = first_batch['question']
true_answers = first_batch['answer']
model_answer = first_batch['model_answer']
retrived_docs = first_batch['retrived_doc']
true_refs = first_batch['ref']
doc_ids = first_batch['doc_id']
dial_ids = first_batch['dial_id']
true_ref_position = first_batch['true_ref_position']
# retrieved_doc_s = first_batch['retrieved_doc_s']
ref_strings = first_batch['ref_string']

In [4]:
# print len of each batch
print('len of questions: ', len(questions))
print('len of true_answers: ', len(true_answers))
print('len of model_answer: ', len(model_answer))
print('len of retrived_docs: ', len(retrived_docs))
print('len of true_refs: ', len(true_refs))
print('len of true_ref_position: ', len(true_ref_position))
# print('len of retrieved_doc_s: ', len(retrieved_doc_s))
print('len of ref_strings: ', len(ref_strings))

len of questions:  16
len of true_answers:  16
len of model_answer:  16
len of retrived_docs:  16
len of true_refs:  16
len of true_ref_position:  16
len of ref_strings:  16


In [16]:
retrived_docs[0].split('\n')

['1. Forgetting to Update Address',
 'sufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the police officer writing you a ticket. If you fail to keep your address current , you will miss a suspension order and may',
 'possible mail correspondence can reach you. Also , turning in your plates is important to avoid an insurance lapse.',
 'By statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not']

In [13]:
retrieved_doc_s

[tensor([ 300,  577, 1650, 1650, 1650, 4214, 1650, 2782, 2287, 2044, 1401,   27,
           82,  758,  809, 3106]),
 tensor([ 577,  300, 1768, 2536, 2536, 2536, 1768, 1650, 3163, 1650, 1151, 1220,
         3683,   27,  758, 3189]),
 tensor([1650, 1401, 2782, 4214, 4214,  577, 2536, 2536, 2536, 3412,  577, 1904,
         3106, 3106, 3436, 4199]),
 tensor([ 333,  333, 2536,  577,  577, 3412, 2044, 1768, 1796, 2287, 2287, 3106,
         3189, 1055, 1947,   82])]

In [50]:
true_ref_position[0]

'[[(333, 346), (346, 416), (416, 567)], [(743, 786)]]'

In [54]:
true_refs[0]

"[[{'sp_id': '5', 'label': 'solution'}, {'sp_id': '6', 'label': 'solution'}, {'sp_id': '7', 'label': 'solution'}], [{'sp_id': '9', 'label': 'precondition'}]]"

In [58]:
retrieved_doc_position

[[tensor([ 300,  577, 1650, 1650]), tensor([ 330,  825, 1764, 1764])],
 [tensor([ 577,  300, 1768, 2536]), tensor([ 825,  330, 1793, 2780])],
 [tensor([1650, 1401, 2782, 4214]), tensor([1764, 1648, 2868, 4461])],
 [tensor([ 333,  333, 2536,  577]), tensor([ 575,  575, 2780,  825])]]

In [61]:
retrived_docs[3].split('\n')

['possible mail correspondence can reach you. Also , turning in your plates is important to avoid an insurance lapse.',
 'that you have an insurance problem , and we will suspend your registration and license. Make sure you turn in your vehicle s license plates at DMV before you cancel your insurance policy. Insurance policies must be from a company licensed in New',
 'a lost title, paying a DRA or scheduling a road test. Our award - winning website is recognized as one of the best in the nation. It has all the answers you need to efficiently perform any DMV transaction. Consider signing up for our MyDMV service,',
 'sufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the police officer writing you a ticket. If you fail to keep your address current , you will miss a suspension order and may']

In [7]:
import re
def get_ref(doc, doc2dial_doc):
    refs_ID = re.findall(r"\d+", doc['references'])
    refs_ID = [int(i) for i in refs_ID]
    sp_list = [doc2dial_doc['doc_data'][doc['domain']][doc['doc_id']]['spans'][str(i)] for i in refs_ID]
    return sp_list

In [35]:
true_answers

["Don't forget to that, there are branches you can go to do this.\nPlease refrence local Maps or our website to find a local branch.",
 'Please go to a local branch and update your address.',
 "You'll need to either bring a copy of your current, new, insurance or pay the un-insured fee.\nPlease make sure to bring all documentation needed to a local branch to expedite this process.",
 'If you need the forms mailed to you, or to download them, please check our website.']

In [9]:
true_refs[0]

"[[{'sp_id': '5', 'label': 'solution'}, {'sp_id': '6', 'label': 'solution'}, {'sp_id': '7', 'label': 'solution'}], [{'sp_id': '9', 'label': 'precondition'}]]"

In [40]:
retrived_docs

['1. Forgetting to Update Address\nsufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the police officer writing you a ticket. If you fail to keep your address current , you will miss a suspension order and may\npossible mail correspondence can reach you. Also , turning in your plates is important to avoid an insurance lapse.\nBy statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not',
 'sufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the police officer writing you a ticket. If you fail to keep your address current , you will miss a suspension order and may\n1. Forgetting to Update Address\nyour license plates to NYS when you get to your new home state. A

In [7]:
string_value = true_refs[0]

In [9]:
import ast
# data_list = ast.literal_eval(string_value)

In [13]:
data_list

[[{'sp_id': '5', 'label': 'solution'},
  {'sp_id': '6', 'label': 'solution'},
  {'sp_id': '7', 'label': 'solution'}],
 [{'sp_id': '9', 'label': 'precondition'}]]

In [14]:
[[term['sp_id'] for term in _] for _ in data_list]

[['5', '6', '7'], ['9']]

In [15]:
def find_ref_id(string_value):
    s = ast.literal_eval(string_value)
    idx_list = [[term['sp_id'] for term in _] for _ in ast.literal_eval(string_value)]
    return idx_list

[find_ref_id(i) for i in true_refs][0]

[['5', '6', '7'], ['9']]

In [16]:
for true_ref in true_refs:
    print(find_ref_id(true_ref))

[['5', '6', '7'], ['9']]
[['35', '36', '37']]


In [4]:
import json
file_path = 'data/doc2dial/doc2dial_doc.json'
with open(file_path, 'r') as f:
    data = json.load(f)

In [8]:
doc_id = doc_ids[0]
refs_ID = [[term['sp_id'] for term in _] for _ in ast.literal_eval(string_value)]
doc_file_span = data['doc_data']['dmv'][doc_id]['spans']
ll = [[doc_file_span[i] for i in l] for l in refs_ID]

In [9]:
true_refs[0]

"[[{'sp_id': '5', 'label': 'solution'}, {'sp_id': '6', 'label': 'solution'}, {'sp_id': '7', 'label': 'solution'}], [{'sp_id': '9', 'label': 'precondition'}]]"

In [10]:
refs_ID

[['5', '6', '7'], ['9']]

In [14]:
ref_string_list = [[term['text_sp'] for term in _] for _ in ll]

In [44]:
' '.join(string for sublist in ref_string_list for string in sublist)

'By statute ,  you must report a change of address to DMV within ten days of moving.  That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ.  If you fail to keep your address current , '

In [43]:
ref_string_list

[['By statute , ',
  'you must report a change of address to DMV within ten days of moving. ',
  'That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. '],
 ['If you fail to keep your address current , ']]

In [29]:
s = [' '.join(_) for _ in ref_string_list]

In [35]:
s

['By statute ,  you must report a change of address to DMV within ten days of moving.  That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. ',
 'If you fail to keep your address current , ']

In [42]:
' '.join([' '.join(_) for _ in ref_string_list])

'By statute ,  you must report a change of address to DMV within ten days of moving.  That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ.  If you fail to keep your address current , '

In [36]:
temp = ''
for ss in s:
    temp += ' ' + ss

temp

' By statute ,  you must report a change of address to DMV within ten days of moving.  That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ.  If you fail to keep your address current , '

In [13]:
retrived_docs[0]

'1. Forgetting to Update Address\nsufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the police officer writing you a ticket. If you fail to keep your address current , you will miss a suspension order and may\npossible mail correspondence can reach you. Also , turning in your plates is important to avoid an insurance lapse.\nBy statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not'

In [51]:
[[(term['start_sp'], term['end_sp']) for term in _] for _ in ll]

[[(333, 346), (346, 416), (416, 567)], [(743, 786)]]

In [6]:
retrived_docs[1]

'By statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not sufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the police officer writing you a ticket. If you fail to keep your address current , you will miss a suspension order and may be'

In [ ]:
def compare(true_ref, retrived_doc):
    pass

In [13]:
p = "Many DMV customers make easily avoidable mistakes that cause them significant problems, including encounters with law enforcement and impounded vehicles. Because we see customers make these mistakes over and over again , we are issuing this list of the top five DMV mistakes and how to avoid them. \n\n1. Forgetting to Update Address \nBy statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not sufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the police officer writing you a ticket. If you fail to keep your address current , you will miss a suspension order and may be charged with operating an unregistered vehicle and/or aggravated unlicensed operation, both misdemeanors. This really happens , but the good news is this is a problem that is easily avoidable. Learn more about how to change the address on your license and registrations [1 ] \n\n2. Leaving the State Without Notifying DMV \nStates communicate with each other , so when you move to another state, be sure to tie up any loose ends regarding your New York State license or registration. That means resolving any unanswered tickets, suspensions or revocations, and surrendering your license plates to NYS when you get to your new home state. A license suspension or revocation here could mean that your new home state will not issue you a license there. Remember , it is important to notify DMV of your new address so that any possible mail correspondence can reach you. Also , turning in your plates is important to avoid an insurance lapse. \n\n3. Letting Insurance Lapse \nBecause we all pay indirectly for crashes involving uninsured motorists , New York State requires every motorist to maintain auto insurance every single day a vehicle is registered. DMV works with insurance companies to electronically monitor your insurance coverage , and we know when coverage is dropped for any reason. When that happens , we mail you an insurance inquiry letter to allow you to clear up the problem. We send 500,000 inquiry letters a year. If the inquiry letter does not resolve the problem , we must suspend the vehicle registration and , if it persists, your driver license!We suspend 300,000 registrations a year for failure to maintain insurance. If you fail to maintain an updated address with us , you won t learn that you have an insurance problem , and we will suspend your registration and license. Make sure you turn in your vehicle s license plates at DMV before you cancel your insurance policy. Insurance policies must be from a company licensed in New York State. Learn more about Insurances Lapes [2] and How to Surrender your Plates [3 ] \n\n4. Understanding how Much Traffic Points Cost \nDMV maintains a point system to track dangerous drivers. Often , motorists convicted of a traffic ticket feel they have resolved all their motoring issues with the local court, but later learn that the Driver Responsibility Assessment DRA is a separate DMV charge based on the total points they accumulate. The $300 DRA fee can be paid in $100 annual installments over three years. Motorists who fail to maintain an updated address with DMV may resolve their tickets with the court, but never receive their DRA assessment because we do not have their new address on record. Failure to pay the DRA will result in a suspended license. Learn more about About the NYS Driver Point System [4] and how to Pay Driver Responsibility Assessment [5 ] \n\n5. Not Bringing Proper Documentation to DMV Office \nAbout ten percent of customers visiting a DMV office do not bring what they need to complete their transaction, and have to come back a second time to finish their business. This can be as simple as not bringing sufficient funds to pay for a license renewal or not having the proof of auto insurance required to register a car. Better yet , don t visit a DMV office at all, and see if your transaction can be performed online, like an address change, registration renewal, license renewal, replacing a lost title, paying a DRA or scheduling a road test. Our award - winning website is recognized as one of the best in the nation. It has all the answers you need to efficiently perform any DMV transaction. Consider signing up for our MyDMV service, which offers even more benefits. Sign up or log into MyDMV [6 ] "

In [29]:
true_answers[0]

"Don't forget to that, there are branches you can go to do this.\nPlease refrence local Maps or our website to find a local branch."

In [5]:
true_ref_position[0]

'[[(333, 346), (346, 416), (416, 567)], [(743, 786)]]'

In [32]:
retrived_docs[0]

'1. Forgetting to Update Address'

In [56]:
sentence = 'By statute , you must'

start = p.find(sentence)
end = start + len(sentence) - 1
print(start, end)

333 353


In [29]:
retrived_docs

['1. Forgetting to Update Address',
 'By statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not sufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the police officer writing you a ticket. If you fail to keep your address current , you will miss a suspension order and may be',
 'possible mail correspondence can reach you. Also , turning in your plates is important to avoid an insurance lapse.',
 'possible mail correspondence can reach you. Also , turning in your plates is important to avoid an insurance lapse.']

In [41]:
sentences = retrived_docs[0].split('\n')

starts = [p.find(sentence) for sentence in sentences]
ends = [start + len(sentence) - 1 for start, sentence in zip(starts, sentences)]
ranges = [(start, end) for start, end in zip(starts, ends)]
print(ranges)

[(300, 330), (577, 825), (1650, 1764), (333, 575)]


In [49]:
retrieved_doc_position[0]

[tensor([ 300,  577, 1650, 1650]), tensor([ 330,  825, 1764, 1764])]

In [3]:
from evaluation import compute_f1

In [5]:
compute_f1(ref_strings[0], retrived_docs[0], return_prec_recall=True)

(0.489795918367347, 0.8, 0.35294117647058826)

In [66]:
true_refs[0]

"[[{'sp_id': '5', 'label': 'solution'}, {'sp_id': '6', 'label': 'solution'}, {'sp_id': '7', 'label': 'solution'}], [{'sp_id': '9', 'label': 'precondition'}]]"

In [12]:
def fn(a, b, *args, **kwargs):
    test = kwargs.get('test_mode', False)
    
    print(args[0])

test_mode = True
fn(1, 2, 3, 4, 5, test_mode=True)

3


In [10]:
import pandas as pd
df = pd.read_csv('data/doc2dial/eval_test.csv')
df.head()

,question,answer,model_answer,ref,retrived_doc,answer_f1,answer_prec,answer_recall,autoais_retrevied,att_f1,att_prec,att_recall,autoais_true_answer
0,What can I do if I forgot to update my address?,"Don't forget to that, there are branches you c...",report a change of address to DMV,"By statute , you must report a change of addr...","1. Forgetting to Update Address\nBy statute , ...",0.066667,0.166667,0.041667,1.0,0.382979,0.251748,0.800000,0.0
1,What if iI already moved?\nPlease go to a loca...,Please go to a local branch and update your ad...,iii.,you won t learn that you have an insurance pro...,"By statute , you must report a change of addre...",0.000000,0.000000,0.000000,NaN,0.117117,0.069519,0.371429,NaN
2,What if my insurance lapsed too?,You'll need to either bring a copy of your cur...,(iv),Because we all pay indirectly for crashes invo...,possible mail correspondence can reach you. Al...,0.000000,0.000000,0.000000,0.0,0.185430,0.126126,0.350000,0.0
3,I'll make sure to do so.,"If you need the forms mailed to you, or to dow...",I will report my change of address to DMV,we mail you an insurance inquiry letter to all...,possible mail correspondence can reach you. Al...,0.083333,0.111111,0.066667,NaN,0.057692,0.030769,0.461538,NaN
4,I'll make sure to do so.,Thank you; it will save you at least an hour i...,I will report my change of address to DMV,you won t learn that you have an insurance pro...,possible mail correspondence can reach you. Al...,0.100000,0.111111,0.090909,NaN,0.113043,0.066667,0.371429,NaN


In [6]:
ans_f1 = []
ans_prec = []
ans_recall = []
for i in range(8):
    f1, prec, recall = evaluation.compute_f1(model_answer[i], true_answers[i], return_prec_recall=True)
    ans_f1.append(f1)
    ans_prec.append(prec)
    ans_recall.append(recall)

In [7]:
ans_f1

[0, 0, 0, 0, 0.11764705882352942, 0, 0.1142857142857143, 0]

In [9]:
questions

['What can I do if I forgot to update my address?',
 'What if iI already moved?\nPlease go to a local branch with a piece of mail and update your address.',
 'What if my insurance lapsed too?',
 "I'll make sure to do so.",
 "I'll make sure to do so.",
 "What if it doesn't solve my problem though?",
 'What will happen if I let my insurance lapse?',
 'No, my insurance is not lapsed right now.\nHow do I learn more about insurance lapses and how to surrender my plates?']

In [19]:
'\n'.join(['a', 'b', 'c']).split('\n')

['a', 'b', 'c']

In [3]:
import pandas as pd
df = pd.read_csv('data/doc2dial/doc2dial_testset/doc2dial_1000_top1/eval.csv')

In [28]:
def fn(cards):
    min_adds = 0
    max_card = max(cards)
    res = 2

    for num in cards:
        if num % 2 == 1:
            min_adds += 1

    for i in range(2, min_adds):
        temp = 0
        for num in cards:
            while num % i != 0:
                temp += 1
                num += 1
        min_adds = min(min_adds, temp)
        res = i
    return res, min_adds

In [35]:
cards = [6, 9, 12]
fn(cards)

(2, 1)

In [1]:
import os
import pandas as pd


In [5]:
path = 'data/doc2dial/Train_dataset'

for subfolder in os.listdir(path):
    subfolder_path = os.path.join(path, subfolder)
    if os.path.isdir(subfolder_path):
        for file in os.listdir(subfolder_path):
            file_path = os.path.join(subfolder_path, file)
            if file.endswith('.csv'):
                df = pd.read_csv(file_path)
                print(file, len(df))


doc2dial_1000_top1_withRefs.csv 4147
DEFAULT_withModelAnswer.csv 4147
DEFAULT_withRefs.csv 4147
doc2dial_500_top5_withModelAnswer.csv 4147
doc2dial_500_top5_withRefs.csv 4147
doc2dial_500_top1_new_withRefs.csv 4147
doc2dial_500_top5_new_withModelAnswer.csv 4147
doc2dial_500_top5_new_withRefs.csv 4147
doc2dial_1000_top1_new_withRefs.csv 4147
doc2dial_1000_top1_new_withModelAnswer.csv 4147


In [6]:
df = pd.read_csv('data/doc2dial/qa_train_dmv_withRefs.csv')
len(df)

4147